# data cleaning for the cashierpaymemt.csv

In [14]:
import pandas as pd
file_path = 'rawdata/cashierpayment.csv'
# Load CSV file into a pandas DataFrame
df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')

it means the cashierpayment.csv is formatted .
## check data consistance inside the table

In [15]:
file_path = 'rawdata/cashierpayment.csv'
# check if every row has same number of columns
with open(file_path, 'r') as f:
    lines = f.readlines()

# Get the number of columns in the header
header_columns = len(lines[0].split(','))
print(lines[0])
print(lines[1])

# Separate the lines into correct and incorrect lines
correct_lines = []
incorrect_lines = []
incorrect_lines.append(lines[0])


# Check each line
for i, line in enumerate(lines):
    num_columns = len(line.split(','))
    if num_columns != header_columns:
        print(f'Row {i} has  {num_columns} of columns instead of {header_columns}')
        incorrect_lines.append(line)
        print(line)
    else:
        correct_lines.append(line)

# Write the correct lines to a new file
correct_file_path = 'format_data/cashierpayment_final.csv'
with open(correct_file_path, 'w') as f:
    # print how many lines are written
    print(f'Writing {len(correct_lines)} line in correct file')
    f.writelines(correct_lines)

# Write the incorrect lines to a new file
incorrect_file_path = 'format_data/cashierpayment-incorrect-lines.csv'
with open(incorrect_file_path, 'w') as f:
    # print how many lines are written
    print(f'Writing {len(incorrect_lines)} line in incorrect file')
    f.writelines(incorrect_lines)



id,noTrans,paidType,cardInit,cardName,cardNumber,totalpaid,MachineName,CompanyCharge,CustomerCharge,ReffNo,ProgressiveDisc

62896,01SCS19E150058,BANK TRANSFER,-,-,-,270000,-,0,0,-,0

Row 932 has  11 of columns instead of 12
61785,01SCS19E070078,CASH,-,-,-,94500,-,0,0,-

Row 6767 has  11 of columns instead of 12
68275,01SCS19E310181,CASH,-,-,-,135000,-,0,0,-

Writing 8008 line in correct file
Writing 3 line in incorrect file


this two line all missing one value so we just delete them.  
## check depulicant/invalid value

In [1]:
import pandas as pd
# Load the CSV file into a pandas DataFrame
correct_file_path = 'format_data/cashierpayment_final.csv'
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# check if there are any null values rows
null_data = df[df.isnull().any(axis=1)]
print(null_data)

Empty DataFrame
Columns: [id, noTrans, paidType, cardInit, cardName, cardNumber, totalpaid, MachineName, CompanyCharge, CustomerCharge, ReffNo, ProgressiveDisc]
Index: []


In [16]:
import pandas as pd

def show_min_max(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numerical values
        if pd.api.types.is_numeric_dtype(df[column]):
            # Find the minimum and maximum values in the column
            min_val = df[column].min()
            max_val = df[column].max()
            # Print the column name with the corresponding min and max values
            print(f'{column}:\n Min value: {min_val}, Max value: {max_val}')

# Load the CSV file into a pandas DataFrame
correct_file_path = 'format_data/cashierpayment_final.csv'
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# Call the function to show the min and max values for each column
show_min_max(df)


id:
 Min value: 60634, Max value: 68642
totalpaid:
 Min value: -3300, Max value: 4401000
CompanyCharge:
 Min value: 0, Max value: 0
CustomerCharge:
 Min value: 0, Max value: 0
ProgressiveDisc:
 Min value: 0, Max value: 0


## remove negative value rows
the totalpaid column has negative value. we need remove this row.

In [17]:
import pandas as pd

correct_file_path = 'format_data/cashierpayment_final.csv'

# Load CSV file into a pandas DataFrame
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# Find rows with negative values in 'totalpaid' column
negative_rows = df[df['totalpaid'] < 0]

# Print the rows with negative values
if len(negative_rows) > 0:
    print("Rows with negative values in 'totalpaid' column:")
    print(negative_rows)
    # Drop rows with negative values
    df = df[df['totalpaid'] >= 0]
    
    # Save the changes to the CSV file
    df.to_csv(correct_file_path, index=False)

else:
    print('No negative values in the DataFrame')

df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

Rows with negative values in 'totalpaid' column:
         id         noTrans paidType cardInit cardName cardNumber  totalpaid  \
6919  68439  01SCS19E310345     CASH        -        -          -      -3300   

     MachineName  CompanyCharge  CustomerCharge ReffNo  ProgressiveDisc  
6919           -              0               0      -                0  


In [18]:
import pandas as pd
correct_file_path = 'format_data/cashierpayment_final.csv'
# Load CSV file into a pandas DataFrame
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

count = df.groupby('id').size()
print('the number of duplicated id: ', count[count > 1].count())
duplicate_rows_id = df[df.duplicated(['id'], keep=False)]
print(duplicate_rows_id)

print('\n===================\n')
count = df.groupby('noTrans').size()
print('the number of duplicated transactions: ', count[count > 1].count())
duplicate_rows_noTrans = df[df.duplicated(['noTrans'], keep=False)]
print(duplicate_rows_noTrans)



the number of duplicated id:  0
Empty DataFrame
Columns: [id, noTrans, paidType, cardInit, cardName, cardNumber, totalpaid, MachineName, CompanyCharge, CustomerCharge, ReffNo, ProgressiveDisc]
Index: []


the number of duplicated transactions:  1
         id         noTrans paidType cardInit        cardName  \
552   61322  01SCS19E040128     CASH        -               -   
7276  61323  01SCS19E040128    DEBIT      MAT  MAESTRO DEBIT    

               cardNumber  totalpaid MachineName  CompanyCharge  \
552                     -          0           -              0   
7276  2182-2598-7651-4092     220500        CBA               0   

      CustomerCharge ReffNo  ProgressiveDisc  
552                0      -                0  
7276               0    114                0  


## remove the duplicant value
we can find that there are two rows share the same noTrans value. we need remove one row

In [19]:
import pandas as pd

correct_file_path = 'format_data/cashierpayment_final.csv'
# Load CSV file into a pandas DataFrame
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# Count the number of duplicated transactions
count = df.groupby('noTrans').size()
print('The number of duplicated transactions:', count[count > 1].count())
duplicate_rows_noTrans = df[df.duplicated(['noTrans'], keep=False)]
print(duplicate_rows_noTrans)


# Remove duplicate values in the 'noTrans' column
df_unique_noTrans = df.drop_duplicates(subset='noTrans')

# Save the changes to the CSV file
df_unique_noTrans.to_csv(correct_file_path, index=False)


The number of duplicated transactions: 1
         id         noTrans paidType cardInit        cardName  \
552   61322  01SCS19E040128     CASH        -               -   
7276  61323  01SCS19E040128    DEBIT      MAT  MAESTRO DEBIT    

               cardNumber  totalpaid MachineName  CompanyCharge  \
552                     -          0           -              0   
7276  2182-2598-7651-4092     220500        CBA               0   

      CustomerCharge ReffNo  ProgressiveDisc  
552                0      -                0  
7276               0    114                0  


# the cashierpayment.csv has been cleaned